# Data Ingestion and Cleaning

In this notebook, we will create a pipeline to:
* Ingest the data 
* Format the data into useable types for ML
* Clean the data - deal with messy/missing data
* Normalize/Standardize the data as needed - get the data into a more efficient set of values

First, we will look at each of the pieces of the pipeline individually, followed by a pipeline to make this easier in the future.

## Part 1.  Data Ingestion

There are many formats we could use to ingest data.  I don't have an API key for my brokerage account, so I have to resort to downloading the data in a fairly messy, manual manner.  The easiest route results in a .csv file, so we will use that for the time being.  

Fortunately, it is not a huge dataset, so does not need to utilize any technologies for big data, such as Spark, or a database backend.

In the future, I would like to create a backend chron job to pull data using an API key, but Chase is not responding to my request for one. 

In [ ]:
# Data Ingestion

